In [58]:
import pandas as pd
import numpy as np
import pickle

import requests
from bs4 import BeautifulSoup

from pymongo import MongoClient
from datetime import datetime
import time
import re as re


In [24]:
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print (f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

#### Notes:

Senate Bills: S.3468 is last

House Bills: H.R.6296 is last

-


### Aggregating Pages

In [3]:
# list of all bills - 113th and 114th congress(es)?
# # https://www.congress.gov/search?pageSize=250&q=%7B%22source%22%3A%22legislation%22%2C%22congress%22%3A%5B%22114%22%2C%22113%22%5D%2C%22type%22%3A%22bills%22%7D&page=1
#250 per page, 18669 bills - this comes to 75 pages

urls = []
for i in range (1,76):
    to_add = 'https://www.congress.gov/search?pageSize=250&q={%22source%22:%22legislation%22,%22congress%22:[%22114%22,%22113%22],%22type%22:%22bills%22}&page='+str(i)
    urls.append(to_add)

print (len(urls))
print (urls[74])

75
https://www.congress.gov/search?pageSize=250&q={%22source%22:%22legislation%22,%22congress%22:[%22114%22,%22113%22],%22type%22:%22bills%22}&page=75


In [25]:
# general use function
def all_html(soup):
    try:
        text = soup
        return text
    except:
        return None
    
    # tuning this and a converter to get a single function
def bill_links_from_search(soup):
    try:
        # this gets us a 250-length list of spans
        text = soup.find_all(class_="result-heading")
        # removes duplicates
        text = list(set(text))
        return text
    except:
        return None

# helper/formatting function to later map to bill_links_from_search above
def url_converter(x):
    x = str(x)
    url_core_regex = r'(https://www.congress.gov/bill/11.*)?r='
    result = re.findall(url_core_regex, x)
    result = result[0]
    result = result[:-1]
    result = str(result) + str('/text?format=txt')
    return result


In [ ]:
# ok lets try running a cleanup once we've already grabbed an initial set...
print (len(urls))
all_texts = []
for i in range(0,len(urls)):
    response = requests.get(urls[i])
    page = response.text
    soup = BeautifulSoup(page)
    all_texts.append(bill_links_from_search(soup))
    time.sleep(.2)
    print (len(all_texts))

75


/home/ubuntu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [6]:
all_texts_flattened = [item for sublist in all_texts for item in sublist]
# print (len(all_texts_flattened)) # should be ~250
# print (all_texts_flattened)
cleaned_list = list(map(url_converter,all_texts_flattened))
print (len(cleaned_list))

6250


#### Never actually got the above working for the full data set on EC2; ended up running it locally and creating a pickle file for the results

'textpage_urls.pkl'

### Individual Bill Page Functions

In [ ]:
# individual text pages of bills example:
# https://www.congress.gov/bill/113th-congress/senate-bill/9/text?format=txt

In [26]:

# these are to be run on an individual bill_text page
def title(soup):
    try:
        text = soup.find(class_="legDetail")
        text = str(text)
        potential_title_regex = r'>((?:S|H).[^<]*)<'
        return re.findall(potential_title_regex, text)[0]
    except:
        return None
    
def sponsor(soup):
    try:
        text = soup.find(text=re.compile('Sponsor'))
        text = text.findParent().findNextSibling().findChild()
        text = str(text)
        sponsor_regex = r'>([^<]*)<'
        return text
        return re.findall(sponsor_regex, text)[0]
    except:
        return None
    
def sponsor_party(soup):
    try:
        text = soup.find(text=re.compile('Sponsor'))
        text = text.findParent().findNextSibling().findChild()
        text = str(text)
        sponsors_regex = r'((?:D|R)-\S*)]'
        if re.findall(sponsors_regex, text)[0][:1] == "D":
            return "Democrat"
        elif re.findall(sponsors_regex, text)[0][:1] == "R":
            return "Republican"
        elif re.findall(sponsors_regex, text)[0][:1] != "D" or re.findall(sponsors_regex, text)[0][:1] != "R":
            return "Independent"
        else:
            return "-"
    except:
        return None
    
def bill_text(soup):
    try:
        text = soup.find(id="billTextContainer")
        text = str(text)
        return text
    except:
        return None

In [ ]:
url = ['https://www.congress.gov/bill/114th-congress/senate-bill/2418/text?format=txt']

for i in range(len(url)):
    response = requests.get(url[i])
    page = response.text
    soup = BeautifulSoup(page)
    print (all_html(soup))

## Old Version - Compiler Using List/Pickle

### Abandoned due to memory limitations - this approach is much more limited and would require significantly more memory, as well as being less searchable, than a database-style approach

In [15]:
# list_of_urls = pd.read_pickle('textpage_urls.pkl')

In [16]:
# len(list_of_urls)

18669

In [49]:
# # using this function on cleaned_list will run it over 18,000 times.
# # so I have altered it to re-pickle every 500 items added.
# @timefunc
# def scrape_list_of_text_urls(url_lst):
#     master_list = []
#     for i in range(len(url_lst)):
#         response = requests.get(url_lst[i])
#         page = response.text
#         soup = BeautifulSoup(page)
#         document = {"title": title(soup), "sponsor": sponsor(soup), "party": sponsor_party(soup),
#                 "bill_text": bill_text(soup)}
#         time.sleep(.05)
#         bills.insert(document)
# #         raw_html.insert({"all_html":all_html(soup)}) # didn't like this
#         master_list.append(document)
#         if len(master_list) % 500 == 0:
#             series_list = pd.Series(master_list)
#             series_list.to_pickle('all_bills.pkl')
#             print ("master list updated, currently at length " + str(len(master_list))+ "; there are currently " +str(len(url_lst)-len(master_list)) + " remaining.")
#             print ("current time: " + str(time.datetime.now()))
#     return master_list
        

In [48]:
# scrape_list_of_text_urls(list_of_urls)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


KeyboardInterrupt: 

# Alternate Version - ACTUAL USE

#### Some Mongo testing

In [43]:
client = MongoClient()

db = client.fletcher_db  # this creates a database with this name
                            # if one doesn't exist

bills = db.bills # create a collection of bills
# raw_html = db.raw_html

# ...  do stuff, make dict

# embed these into scrape_list_of_urls function, building mongo database?
# bills.insert(document)
# raw_html.insert("all_html":all_html(soup))

In [60]:
# this version ONLY uploads to the same-server's mongo-DB
# might solve the potential memory issue of a 
# extremely long list of dicts (15k+ with large dicts)

# sets up the mongo client
client = MongoClient()
db = client.fletcher_db  # this creates a database with this name if one doesn't exist
bills = db.bills # this creates a collection if one doesn't exist

@timefunc
def database_upload(url_lst):
    for i in range(len(url_lst)):
        response = requests.get(url_lst[i])
        page = response.text
        soup = BeautifulSoup(page)
        # these are the individual values you want from each page, you will need to define document with whatever scraper functions you are using
        document = {"title": title(soup), "sponsor": sponsor(soup), "party": sponsor_party(soup),
                "bill_text": bill_text(soup)}
        time.sleep(.05)
        # adds it to mongo
        bills.insert(document)
        # just a peace-of-mind-loop, will let you know how the scraping is going
        if i % 500 == 0:
            print (str(i) + " documents have been uploaded to the db; " + str(len(url_lst)-i) + " remaining" )
            print ("current time: " + str(datetime.now()))
    return "all done!"

In [61]:
database_upload(list_of_urls)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


0 documents have been uploaded to the db; 18669 remaining
current time: 2016-11-04 03:46:27.442402
500 documents have been uploaded to the db; 18169 remaining
current time: 2016-11-04 03:51:18.262735
1000 documents have been uploaded to the db; 17669 remaining
current time: 2016-11-04 03:57:41.725918
1500 documents have been uploaded to the db; 17169 remaining
current time: 2016-11-04 04:06:40.128120
2000 documents have been uploaded to the db; 16669 remaining
current time: 2016-11-04 04:15:23.099045
2500 documents have been uploaded to the db; 16169 remaining
current time: 2016-11-04 04:24:58.981720
3000 documents have been uploaded to the db; 15669 remaining
current time: 2016-11-04 04:34:47.718739
3500 documents have been uploaded to the db; 15169 remaining
current time: 2016-11-04 04:44:37.196929
4000 documents have been uploaded to the db; 14669 remaining
current time: 2016-11-04 04:53:21.693411
4500 documents have been uploaded to the db; 14169 remaining
current time: 2016-11-04 

ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

### Connection broken, failed at 13531, creating second list to finish scraping

In [76]:
second_list_bit = list_of_urls[13531:].values.copy()
database_upload(second_list_bit)

/home/ubuntu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


0 documents have been uploaded to the db; 5138 remaining
current time: 2016-11-04 15:37:42.085005
500 documents have been uploaded to the db; 4638 remaining
current time: 2016-11-04 15:49:05.945929
1000 documents have been uploaded to the db; 4138 remaining
current time: 2016-11-04 15:58:52.091571
1500 documents have been uploaded to the db; 3638 remaining
current time: 2016-11-04 16:10:20.331423
2000 documents have been uploaded to the db; 3138 remaining
current time: 2016-11-04 16:20:58.209749
2500 documents have been uploaded to the db; 2638 remaining
current time: 2016-11-04 16:30:57.245603
3000 documents have been uploaded to the db; 2138 remaining
current time: 2016-11-04 16:41:30.510345
3500 documents have been uploaded to the db; 1638 remaining
current time: 2016-11-04 16:52:25.337703
4000 documents have been uploaded to the db; 1138 remaining
current time: 2016-11-04 17:02:57.402735
4500 documents have been uploaded to the db; 638 remaining
current time: 2016-11-04 17:13:21.89

'all done!'

### Ok so now this is all in mongo, let's go play.

In [ ]:
# example import of a collection into a dataframe

# client = MongoClient()
# db = client.database_name
# collection = db.collection_name
# data = pd.DataFrame(list(collection.find()))

In [77]:
df = pd.DataFrame(list(bills.find()))

In [81]:
df.shape

(18669, 5)

# Continued Work is in the Analysis Section. If MongoDB for the scraper is on the same computer as the Analysis machine, you can directly read from the DB in an new python notebook, otherwise exporting the DB into a JSON file and going from there works.